In [1]:
!nvidia-smi

Thu Jan 29 16:04:21 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.105.08             Driver Version: 580.105.08     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:1A:00.0 Off |                    0 |
| N/A   33C    P0             44W /  300W |       0MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
import os
import sys
sys.path.append("/home/xucong24/Compiler")
from src.rl.llvm_wrapper import llvm_wrapper, PassformerObservation
from src.model import PassformerModel, PassformerConfig, Inst2VecTokenizer, OptiSeqTokenizer
from datasets import Dataset
import gym
import compiler_gym

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy

In [3]:
from compiler_gym.envs.llvm import make_benchmark

model_id = "/home/xucong24/Compiler/work_dirs/passformer_gallvm_seq2seq_concat/20260120_195517/final_model"
encoder_tokenizer_id = "/home/xucong24/Compiler/checkpoints/Inst2VecTokenizer"
decoder_tokenizer_id = "/home/xucong24/Compiler/checkpoints/OptiSeqTokenizer"

# model = PassformerModel.from_pretrained(model_id)
encoder_tokenizer = Inst2VecTokenizer.from_pretrained(encoder_tokenizer_id)
decoder_tokenizer = OptiSeqTokenizer.from_pretrained(decoder_tokenizer_id)


bc_files = [
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc",
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc", 
    "/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc"
    ]

def prepare_dataset(bc_files, encoder_tokenizer):
    all_data = {
        "llvm_ir": [],
        "autophase": [],    # 原始 autophase 特征
        "bc_path": []       # 用于 Reward 计算
    }
    
    for bc in bc_files:
        # 1. 环境初始化
        env = llvm_wrapper([bc], is_from_bc=True)
        observation = env.reset()
        all_data["llvm_ir"].append(observation.llvm_ir)
        all_data["autophase"].append(observation.autophase)
        all_data["bc_path"].append(bc)
        env.close()
        
    return Dataset.from_dict(all_data)

datasets = prepare_dataset(bc_files, encoder_tokenizer,)
print(datasets)


Dataset({
    features: ['llvm_ir', 'autophase', 'bc_path'],
    num_rows: 10
})


In [4]:
datasets[0]

{'llvm_ir': '; ModuleID = \'-\'\nsource_filename = "-"\ntarget datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-unknown-linux-gnu"\n\n%struct._QITEM = type { i32, i32, i32, %struct._QITEM* }\n%struct._IO_FILE = type { i32, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, %struct._IO_marker*, %struct._IO_FILE*, i32, i32, i64, i16, i8, [1 x i8], i8*, i64, %struct._IO_codecvt*, %struct._IO_wide_data*, %struct._IO_FILE*, i8*, i64, i32, [20 x i8] }\n%struct._IO_marker = type opaque\n%struct._IO_codecvt = type opaque\n%struct._IO_wide_data = type opaque\n%struct._NODE = type { i32, i32 }\n\n@NUM_NODES = dso_local global i32 0, align 4\n@qHead = dso_local global %struct._QITEM* null, align 8\n@g_qCount = dso_local global i32 0, align 4\n@.str = private unnamed_addr constant [4 x i8] c" %d\\00", align 1\n@stdout = external dso_local global %struct._IO_FILE*, align 8\n@stderr = external dso_local global %struct._IO_FILE*, align

In [4]:
def collate_fn(batch):
    return {
        'llvm_ir': [x['llvm_ir'] for x in batch],
        'autophase': torch.stack([torch.tensor(x['autophase']) for x in batch]),
        'bc_path': [x['bc_path'] for x in batch]
    }

loader = DataLoader(datasets, batch_size=2, shuffle=True, collate_fn=collate_fn)
for batch in loader:
    print(batch)

{'llvm_ir': ['; ModuleID = \'-\'\nsource_filename = "-"\ntarget datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"\ntarget triple = "x86_64-unknown-linux-gnu"\n\n%struct._QITEM = type { i32, i32, i32, %struct._QITEM* }\n%struct._IO_FILE = type { i32, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, i8*, %struct._IO_marker*, %struct._IO_FILE*, i32, i32, i64, i16, i8, [1 x i8], i8*, i64, %struct._IO_codecvt*, %struct._IO_wide_data*, %struct._IO_FILE*, i8*, i64, i32, [20 x i8] }\n%struct._IO_marker = type opaque\n%struct._IO_codecvt = type opaque\n%struct._IO_wide_data = type opaque\n%struct._NODE = type { i32, i32 }\n\n@NUM_NODES = dso_local global i32 0, align 4\n@qHead = dso_local global %struct._QITEM* null, align 8\n@g_qCount = dso_local global i32 0, align 4\n@.str = private unnamed_addr constant [4 x i8] c" %d\\00", align 1\n@stdout = external dso_local global %struct._IO_FILE*, align 8\n@stderr = external dso_local global %struct._IO_FILE*, alig

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PassformerModel.from_pretrained(model_id)
model = model.to(device)

In [8]:
def compiler_gym_reward_fn(completions, bc_path, **kwargs):
    """
    completions: 模型生成的 Pass 字符串列表 (e.g., ["-mem2reg -gvn", "-O3"])
    bc_path: 对应的位码文件路径
    """
    rewards = []
    print(completions)
    
    # 注意：GRPO 会批量传入数据
    for completion, path in zip(completions, bc_path):
        try:
            # 1. 创建临时的 CompilerGym 环境
            # 或者使用全局环境重置：env.reset(path)
            temp_env = llvm_wrapper([path], is_from_bc=True)
            observation = temp_env.reset()
            
            # 2. 执行生成的 Pass 序列
            # 假设你的 completion 是 "pass1 pass2"
            _, reward, done, info = temp_env.multistep_by_action_flags(completion)
            
            # 3. 记录奖励（例如使用代码体积变化率）
            rewards.append(float(reward))
            
            temp_env.close()
        except Exception as e:
            print(f"Error processing {path}: {e}")
            rewards.append(-1.0) # 编译失败或崩溃给予惩罚
            
    return rewards

completions = ["-mem2reg -gvn" for i in range(10)]
compiler_gym_reward_fn(completions, bc_files)

['-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn', '-mem2reg -gvn']


[0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625,
 0.8469387755102041,
 0.9765625]

In [ ]:
class GRPOTrainerManual:
    def __init__(self, model, encoder_tokenizer, decoder_tokenizer, reward_fn, 
                 beta=0.01, eps=0.2, lr=1e-5, num_generations=8):
        self.model = model
        # 参考模型，用于计算 KL 散度，保持冻结
        self.ref_model = copy.deepcopy(model).eval()
        self.encoder_tokenizer = encoder_tokenizer
        self.decoder_tokenizer = decoder_tokenizer
        self.reward_fn = reward_fn
        
        self.beta = beta           # KL 惩罚系数
        self.eps = eps             # PPO 剪切阈值
        self.num_gens = num_generations
        self.optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    def get_log_probs(self, logits, tokens):
        """计算生成的 token 的 log 概率"""
        log_probs = F.log_softmax(logits, dim=-1)
        # 选取实际生成的 token 对应的概率
        per_token_logps = torch.gather(log_probs, dim=2, index=tokens.unsqueeze(-1)).squeeze(-1)
        return per_token_logps

    def train_step(self, batch):
        """
        batch: 包含 'llvm_ir' (IR文本), 'autophase', 'bc_path'
        """
        device = self.model.device
        llvm_irs = batch['llvm_ir']
        autophases = batch['autophase'].to(device)
        bc_paths = batch['bc_path']

        # print("llvm_irs", llvm_irs)
        # print("autophases", autophases)
        # print("bc_paths", bc_paths)

        # --- 1. 采样阶段 (Rollout) ---
        self.model.eval()
        with torch.no_grad():
            # 为每个 llvm_ir 生成多组序列
            # 编码输入
            inputs = self.encoder_tokenizer(llvm_irs, padding=True, max_length=128, return_tensors="pt")
            # print("inputs", inputs)
            
            # 生成结果
            # 注意：这里需要重复输入以实现 Batch 生成
            expanded_input_ids = inputs['input_ids'].repeat_interleave(self.num_gens, dim=0).to(device)
            expanded_attention_mask = inputs['attention_mask'].repeat_interleave(self.num_gens, dim=0).to(device)
            expanded_autophase = autophases.repeat_interleave(self.num_gens, dim=0).to(device)
            
            output_ids = self.model.generate(
                input_ids=expanded_input_ids,
                attention_mask=expanded_attention_mask,
                autophase=expanded_autophase,
                max_length=20,
                do_sample=True # 必须开启采样以保证多样性
            )
            # 提取生成的部分 (假设 output_ids 包含 llvm_ir，需切片；若模型直接返回 completion 则不用)
            completions = output_ids 
            # print("completions", completions)
            
            # 解码为文本以获取 Reward
            completion_texts = self.decoder_tokenizer.batch_decode(completions, skip_special_tokens=True)
            # print("completion_texts", completion_texts)

        # --- 2. 奖励计算 (Reward) ---
        # 扩展 bc_paths 以匹配生成数量
        expanded_bc_paths = [p for p in bc_paths for _ in range(self.num_gens)]
        # print("expanded_bc_paths", expanded_bc_paths)
        rewards = self.reward_fn(completion_texts, expanded_bc_paths)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        # print("rewards", rewards)

        # --- 3. 计算优势 (Advantage) ---
        # 按组归一化：(r - mean) / std
        rewards_reshaped = rewards.view(-1, self.num_gens)
        mean_r = rewards_reshaped.mean(dim=1, keepdim=True)
        std_r = rewards_reshaped.std(dim=1, keepdim=True) + 1e-8
        advantages = ((rewards_reshaped - mean_r) / std_r).view(-1)

        # --- 4. 优化阶段 (Update) ---
        self.model.train()
        # 计算当前模型的 log_probs
        outputs = self.model(
            input_ids=expanded_input_ids,
            attention_mask=expanded_attention_mask,
            autophase=expanded_autophase,
            # labels=completions # 强制模型计算这些 token 的 logits
            decoder_input_ids=completions,          # 明确指定 Decoder 输入
            labels=completions                      # 用于计算 logits 和 loss
        )
        current_log_probs = self.get_log_probs(outputs.logits, completions)
        
        # 计算参考模型的 log_probs (用于 KL)
        with torch.no_grad():
            ref_outputs = self.ref_model(
                input_ids=expanded_input_ids,
                attention_mask=expanded_attention_mask,
                autophase=expanded_autophase,
                decoder_input_ids=completions,          # 明确指定 Decoder 输入
                labels=completions                      # 用于计算 logits 和 loss
            )
            ref_log_probs = self.get_log_probs(ref_outputs.logits, completions)

        # 重要性采样 Ratio (这里简化处理，假设采样时的 log_probs 与 current 近似)
        # 在标准的 GRPO 中，Ratio 是 current / old_sampling
        ratio = torch.exp(current_log_probs.sum(dim=-1) - current_log_probs.detach().sum(dim=-1)) 
        
        # PPO Clip Loss
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1 - self.eps, 1 + self.eps) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()

        # KL Loss (避免偏离参考模型太远)
        kl = torch.exp(ref_log_probs - current_log_probs) - (ref_log_probs - current_log_probs) - 1
        kl_loss = self.beta * kl.mean()

        total_loss = policy_loss + kl_loss

        self.optimizer.zero_grad()
        total_loss.backward()
        self.optimizer.step()

        return total_loss.item(), rewards.mean().item()

In [9]:
my_trainer = GRPOTrainerManual(
    model=model,
    encoder_tokenizer=encoder_tokenizer,
    decoder_tokenizer=decoder_tokenizer,
    reward_fn=compiler_gym_reward_fn,
    num_generations=8
)

# 3. 训练循环
for epoch in range(3):
    pbar = tqdm(loader)
    for batch in pbar:
        loss, avg_reward = my_trainer.train_step(batch)
    pbar.set_description(f"Loss: {loss:.4f} | Reward: {avg_reward:.4f}")

  0%|          | 0/5 [00:00<?, ?it/s]

autophases tensor([[  0,   0,  16,  12,   2,  16,   8,   2,   4,   8,   0,   0,   0,  29,
           0,  24,   9,   2,  32,  44,  41,  14,  36,  16,  13,   0,   5,  26,
           3,   5,  24,  20,  24,  33,   5,  10,   3,  51,   0,   1,   0,   5,
           0,   0,   0,  42,   0,   1,   8,   5,  29, 242, 157,  15,   0, 103],
        [  0,   0,  36,  26,   6,  30,  15,   3,   8,  18,   0,   0,   0,  57,
           0,  48,  18,   6,  66, 105,  26,  41,  48,  30,  20,   0,  20,  23,
           0,   8,  49,  26,  48,  55,  36,  18,   0, 132,   8,   0,   0,   6,
          16,   0,   0,  57,   0,   0,   0,   0,  57, 450, 303,  17,   0, 197]],
       device='cuda:0')
bc_paths ['/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc', '/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc']
inputs {'input_ids': tensor([[8566, 8564, 8564, 8564, 8564,    5,   46,   46,   40, 8564,   13,    8,
         8564, 1348,  178,  286,  214,  182,  235,  697, 1513,  1

/home/xucong24/Compiler/.venv/lib/python3.10/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(


rewards tensor([-0.0078, -0.1328, -0.0078, -0.0781,  0.3359,  0.2969,  0.3594,  0.2969,
        -0.7245,  0.2296, -0.1327,  0.0357,  0.0000, -0.0612,  0.0357, -0.7806],
       device='cuda:0')


 20%|██        | 1/5 [01:30<06:00, 90.02s/it]

...

autophases tensor([[  0,   0,  16,  12,   2,  16,   8,   2,   4,   8,   0,   0,   0,  29,
           0,  24,   9,   2,  32,  44,  41,  14,  36,  16,  13,   0,   5,  26,
           3,   5,  24,  20,  24,  33,   5,  10,   3,  51,   0,   1,   0,   5,
           0,   0,   0,  42,   0,   1,   8,   5,  29, 242, 157,  15,   0, 103],
        [  0,   0,  36,  26,   6,  30,  15,   3,   8,  18,   0,   0,   0,  57,
           0,  48,  18,   6,  66, 105,  26,  41,  48,  30,  20,   0,  20,  23,
           0,   8,  49,  26,  48,  55,  36,  18,   0, 132,   8,   0,   0,   6,
          16,   0,   0,  57,   0,   0,   0,   0,  57, 450, 303,  17,   0, 197]],
       device='cuda:0')
bc_paths ['/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc', '/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc']
inputs {'input_ids': tensor([[8566, 8564, 8564, 8564, 8564,    5,   46,   46,   40, 8564,   13,    8,
         8564, 1348,  178,  286,  214,  182,  235,  697, 1513,  1

  0%|          | 0/5 [00:00<?, ?it/s]

autophases tensor([[  0,   0,  36,  26,   6,  30,  15,   3,   8,  18,   0,   0,   0,  57,
           0,  48,  18,   6,  66, 105,  26,  41,  48,  30,  20,   0,  20,  23,
           0,   8,  49,  26,  48,  55,  36,  18,   0, 132,   8,   0,   0,   6,
          16,   0,   0,  57,   0,   0,   0,   0,  57, 450, 303,  17,   0, 197],
        [  0,   0,  16,  12,   2,  16,   8,   2,   4,   8,   0,   0,   0,  29,
           0,  24,   9,   2,  32,  44,  41,  14,  36,  16,  13,   0,   5,  26,
           3,   5,  24,  20,  24,  33,   5,  10,   3,  51,   0,   1,   0,   5,
           0,   0,   0,  42,   0,   1,   8,   5,  29, 242, 157,  15,   0, 103]],
       device='cuda:0')
bc_paths ['/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_dijkstra.bc', '/home/xucong24/Compiler/datasets/cbench-v1/benchmark_cbench-v1_crc32.bc']
inputs {'input_ids': tensor([[8566, 8564, 8564, 8564, 8564, 8564, 8564,    8, 8564, 8564,   24, 8564,
         8564,   53, 8564, 8564, 8564, 8564, 8564, 8564,   17, 85

  0%|          | 0/5 [00:13<?, ?it/s]


KeyboardInterrupt: 